In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import k3d

from libs import tpcpath
from torch_point_cloud.utils import pytorch_tools
pytorch_tools.set_seed(0)
device = pytorch_tools.select_device("cuda")


In [3]:
def visualize_pc(pc, colors=None):
    if colors is None:
        colors = np.full(len(pc), 0x000000, dtype=np.float64) # white: 0xFFFFFF
    plot = k3d.plot()
    points = k3d.points(pc, colors.astype(np.float32), point_size=0.04, shader='flat')
    plot += points
    plot.display()


## get point cloud dataset

In [4]:
from libs.dataset import SimpleObjectDataset

In [5]:
dataset = SimpleObjectDataset()
pc, label = dataset[0]
visualize_pc(pc)


/home/coder/anaconda3/envs/py37/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

## get FPS points

In [6]:
%load_ext autoreload
%autoreload 1
%reload_ext autoreload

from torch_point_cloud.modules.functional import furthest_point_sample, index2points

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
t_pc = torch.tensor([pc])
t_pc = t_pc.to(device)
t_pc = t_pc.transpose(1,2)
center_t_idxs = furthest_point_sample(t_pc, 512)
center_t_pc = index2points(t_pc, center_t_idxs)
center_pc = pytorch_tools.t2n(center_t_pc.transpose(1,2)[0])
visualize_pc(center_pc)


/home/coder/anaconda3/envs/py37/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

## get KNN points

In [8]:
from torch_point_cloud.modules.functional import k_nearest_neighbors as knn

In [9]:
knn_idxs, knn_dists = knn(t_pc, center_t_pc, 5)
print("knn_idxs.shape:", knn_idxs.shape)

knn_idxs.shape: torch.Size([1, 512, 5])


## compare C++ with python implementation.

In [10]:
from torch_point_cloud.modules.functional import py_k_nearest_neighbors
from torch_point_cloud.utils.monitor import timecheck

In [11]:
t = timecheck()
knn_idxs, knn_dists = knn(t_pc, center_t_pc, 3)
print("c++ impl. shape:", knn_idxs.shape)
t = timecheck(t, "c++ impl. time")

c++ impl. shape: torch.Size([1, 512, 3])
c++ impl. time: 0.0004522800445556640625s


In [12]:
t = timecheck()
knn_idxs_py, knn_dists_py = py_k_nearest_neighbors(center_t_pc, t_pc, 3, True)
print("python impl. shape:", knn_idxs_py.shape)
t = timecheck(t, "python impl. time")
print(False in (knn_idxs == knn_idxs_py))

python impl. shape: torch.Size([1, 512, 3])
python impl. time: 0.0013401508331298828125s
False


## compare other implementation.

In [1]:
# PointRCNN impl. from https://github.com/sshaoshuai/PointRCNN
import pointnet2_cuda as pointnet2
from typing import Tuple
class ThreeNN(torch.autograd.Function):

    @staticmethod
    def forward(ctx, unknown: torch.Tensor, known: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Find the three nearest neighbors of unknown in known
        :param ctx:
        :param unknown: (B, N, 3)
        :param known: (B, M, 3)
        :return:
            dist: (B, N, 3) l2 distance to the three nearest neighbors
            idx: (B, N, 3) index of 3 nearest neighbors
        """
        assert unknown.is_contiguous()
        assert known.is_contiguous()

        B, N, _ = unknown.size()
        m = known.size(1)
        dist2 = torch.cuda.FloatTensor(B, N, 3)
        idx = torch.cuda.IntTensor(B, N, 3)

        pointnet2.three_nn_wrapper(B, N, m, unknown, known, dist2, idx)
        return torch.sqrt(dist2), idx

    @staticmethod
    def backward(ctx, a=None, b=None):
        return None, None

three_nn = ThreeNN.apply


ImportError: libc10.so: cannot open shared object file: No such file or directory

In [14]:
t = timecheck()
c_oth_dists, c_oth_idx = three_nn(center_t_pc.transpose(1,2).contiguous(), t_pc.transpose(1,2).contiguous())
# print(c_oth_idx.shape)
t = timecheck(t, "other impl. time")

other impl. time: 0.0008552074432373046875s


## compare implementations using scene data batches.

In [15]:
from libs.dataset import SimpleSceneDataset
from torch.utils.data import DataLoader

In [20]:
def speed_test(method, loader):
    for i, data in enumerate(loader): pass # for speed processing
    
    # print name
    if method == 0:
        t_name = "original c++ impl. time"
    elif method == 1:
        t_name = "original py impl. time"
    elif method == 2:
        t_name = "other c++ impl. time"
    else:
        raise NotImplementedError()

    # timer start
    t = timecheck()
    for _ in range(100):
        for i, data in enumerate(loader):
            point_clouds, sem_labels, ins_labels = data
            point_clouds = point_clouds[:, :3].to(device)
            center_idxs = furthest_point_sample(point_clouds, 1024)
            center_pc = index2points(point_clouds, center_idxs)
            if method == 0:
                _ = knn(center_points, point_clouds, k=3)
            elif method == 1:
                _ = py_k_nearest_neighbors(center_pc, point_clouds, k=3, memory_saving=False)
            elif method == 2:
                _ = three_nn(center_pc.transpose(1,2).contiguous(), point_clouds.transpose(1,2).contiguous())
            else:
                raise NotImplementedError()
    # timer end
    timecheck(t, t_name)
        

In [21]:
dataset = SimpleSceneDataset()
loader = DataLoader(
    dataset,
    batch_size=32,
    num_workers=8,
    pin_memory=True,
    shuffle=False
)

In [27]:
speed_test(0, loader)

original c++ impl. time: 22.93700408935546875s


In [28]:
speed_test(1, loader)

original py impl. time: 22.9990055561065673828125s


In [29]:
speed_test(2, loader)

other c++ impl. time: 23.0654144287109375s
